In [142]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [143]:
# Load the dataset
DATA_PATH = '../data-preprocessing/VinhLong_data.csv'
data = pd.read_csv(DATA_PATH)
data = data[data['Nhiệt độ'] <= 60]

In [144]:
data['Ngày'] = pd.to_datetime(data['Ngày'], format='%d/%m/%Y')
data['Giờ'] = pd.to_datetime(data['Giờ'], format='%H:%M').dt.time
data['Datetime'] = pd.to_datetime(data['Ngày'].astype(str) + ' ' + data['Giờ'].astype(str))
data.set_index('Datetime', inplace=True)
data.drop(['Ngày', 'Giờ'], axis=1, inplace=True)
data.head()

,Nhiệt độ,Độ ẩm,Khí áp,T.độ gió,H. gió
Datetime,,,,,
2014-01-01 01:00:00,22.2,89,1013.7,1.3,225
2014-01-01 02:00:00,21.8,90,1012.9,1,270
2014-01-01 03:00:00,22.8,80,1012.5,3.1,320
2014-01-01 04:00:00,22.7,80,1012.5,1.2,235
2014-01-01 05:00:00,22.0,84,1012.5,3.7,319


In [145]:
data.replace(['-', ''], np.nan, inplace=True)
data.dropna(inplace=True)
data = data.apply(pd.to_numeric, errors='ignore')
data.tail()

,Nhiệt độ,Độ ẩm,Khí áp,T.độ gió,H. gió
Datetime,,,,,
2024-01-15 19:00:00,28.3,65,1011.8,2.2,154
2024-01-15 20:00:00,27.4,70,1012.1,2.1,152
2024-01-15 21:00:00,27.0,72,1012.5,0.2,149
2024-01-15 22:00:00,26.5,75,1012.2,1.3,153
2024-02-21 07:00:00,24.4,97,1012.6,1.4,142


In [146]:
print(data.isna().sum())

Nhiệt độ    0
Độ ẩm       0
Khí áp      0
T.độ gió    0
H. gió      0
dtype: int64


In [147]:
hourly_data = data.resample('H').mean()
hourly_data.replace(['-', ''], np.nan, inplace=True)
hourly_data.dropna(inplace=True)
hourly_data = hourly_data.apply(pd.to_numeric, errors='ignore')
hourly_data.tail()

,Nhiệt độ,Độ ẩm,Khí áp,T.độ gió,H. gió
Datetime,,,,,
2024-01-15 19:00:00,28.3,65.0,1011.8,2.2,154.0
2024-01-15 20:00:00,27.4,70.0,1012.1,2.1,152.0
2024-01-15 21:00:00,27.0,72.0,1012.5,0.2,149.0
2024-01-15 22:00:00,26.5,75.0,1012.2,1.3,153.0
2024-02-21 07:00:00,24.4,97.0,1012.6,1.4,142.0


In [148]:
hourly_data.reset_index(drop=True, inplace=True)

In [149]:
print(hourly_data.isna().sum())

Nhiệt độ    0
Độ ẩm       0
Khí áp      0
T.độ gió    0
H. gió      0
dtype: int64


In [150]:
def train_test_split(data, test_percents):
    n_test = int(len(data) * test_percents)
    return data[:-n_test], data[-n_test:]

In [151]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [152]:
def create_features(data, n_lags=3):
    df = data.copy()
    for n in range(1, n_lags + 1):
        df[f'lag_{n}'] = df['Nhiệt độ'].shift(n)
    # target
    df['target_temp'] = df['Nhiệt độ'].shift(-1)
    df.dropna(inplace=True)
    return df

# Apply feature engineering
hourly_data = create_features(hourly_data, n_lags=3)


In [153]:
# from sklearn.model_selection import GridSearchCV

# def xgboost_grid_search(trainX, trainy):
#     model = XGBRegressor(objective='reg:squarederror')
#     param_grid = {
#         'n_estimators': [100, 500, 1000],
#         'max_depth': [3, 5, 7, 9],
#         'learning_rate': [0.01, 0.05, 0.1]
#     }
#     grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
#     grid_search.fit(trainX, trainy)
#     print("Best parameters found: ", grid_search.best_params_)
#     return grid_search.best_estimator_

In [154]:

def xgboost_forecast(train, testX):
    train = np.asarray(train)
    trainX, trainy = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective='reg:squarederror')
    model.fit(trainX, trainy)
    yhat = model.predict([testX])
    return yhat[0]


In [155]:
from tqdm import tqdm

In [160]:
def rolling_forecast(data, window_size):
    predictions = list()
    train, test = train_test_split(data.values, 0.2)
    history = [x for x in train]
    process = tqdm(range(len(test)))

    for i in process:
        rolling_train = history[-window_size:]
        testX, testy = test[i, :-1], test[i, -1]
        yhat = xgboost_forecast(rolling_train, testX)
        predictions.append(yhat)
        history.append(test[i])
        process.set_description('>expected=%.1f, predicted=%.1f' % (testy, yhat))    
    predictions = np.array(predictions)
    error = np.sqrt(mean_squared_error(test[:, -5:], predictions))

    return error, test[:, -1], predictions


In [161]:
hourly_data.head()

,Nhiệt độ,Độ ẩm,Khí áp,T.độ gió,H. gió,lag_1,lag_2,lag_3,target_temp
3,22.7,80.0,1012.5,1.2,235.0,22.8,21.8,22.2,22.0
4,22.0,84.0,1012.5,3.7,319.0,22.7,22.8,21.8,21.6
5,21.6,85.0,1013.2,2.2,321.0,22.0,22.7,22.8,21.5
6,21.5,83.0,1014.0,1.7,283.0,21.6,22.0,22.7,21.8
7,21.8,83.0,1014.7,2.7,303.0,21.5,21.6,22.0,22.7


In [162]:
print(hourly_data.isna().sum())

Nhiệt độ       0
Độ ẩm          0
Khí áp         0
T.độ gió       0
H. gió         0
lag_1          0
lag_2          0
lag_3          0
target_temp    0
dtype: int64


In [163]:
from sklearn.metrics import mean_squared_error

window_size = 100

# Evaluate the model
rmse, y, yhat = rolling_forecast(hourly_data, window_size)
print('RMSE: %.3f' % rmse)

KeyboardInterrupt: 

In [ ]:
plt.plot(y, label='Expected')
plt.plot(yhat, label='Predicted')
plt.legend()
plt.show()